# Bit Manipulation

* Modify data at the binary level. 
* Faster than integer/flaot operatons. 
* Operators 
    * __AND__: `&` => True if both are true
    * __OR__: `|` => False if both are false
    * __NOT__: `~` => True if false, vice versa 
    * __XOR__: `^` => false for identicaal input, true otherwise. 
    * __LShift__: `<<` => $n<<i = n2^i$ 
    * __RShift__: `>>`  => $n>>i = \lfloor\frac{n}{2^i}\rfloor$

# Examples 

In [14]:
# assigning arbitrary value 
a=5
b=6

print(f'a={bin(a)}, b={bin(b)}')  #binary representation of int
print('-----------------------')
 
print(f'a & b = { a & b}') # bw and
print(f'a | b = { a | b}') # bw or
print(f'a ^ b = { a ^ b}') # bw not
print(f'a>>1 = { a>>1 }')  # left shift 
print(f'a<<1 = { a<<1 }')  # right shift 

a=0b101, b=0b110
-----------------------
a & b = 4
a | b = 7
a ^ b = 3
a>>1 = 2
a<<1 = 10


# 1. Odd even Check 

* __Problem__: Write a function `is_odd(n)` that returns `True` if `n` is odd and `False` otherwise.  

In [20]:
# solution 
def is_odd(n:int)->bool:
    return True if n&1 == 1 else False  # all odd numbers has LSB=1

In [21]:
# Verify
lim = 10 # verify for int [0:lim)
for i in range(lim):
    result = is_odd(i)
    print(i,end=' ')
    print('odd') if result == True else print('evnen')

0 evnen
1 odd
2 evnen
3 odd
4 evnen
5 odd
6 evnen
7 odd
8 evnen
9 odd


# 2. Bit Manipulation
Three types of i-th bit-manipulation operation 
1. Get the i-th bit
2. Set the i-th bit 
3. Clear the i-th bit

## 2.1. Get the $i^{th}$ bit


In [24]:
def get_ith_bit(n:int, i:int) -> bool:
    """Returns the ith bit of n"""
    n>>i       # shift i position 
    return n&1 # return LSB 

In [25]:
n=27
i=3

print(f'{i}-th bit of {n} ({bin(n)}) = {get_ith_bit(n,i)}')

3-th bit of 27 (0b11011) = 1


## 2.2. Set $i^{th}$ bit 

In [35]:
def set_ith_bit(n:int, i:int) -> int:
    "return n by setting the i-th bit of it"
    mask = 1<<i   # 1 folloed by i 0s
    return n|mask

In [36]:
# test 
n = 23
i = 3
print(f'before: {n} = {bin(n)}')

n = set_ith_bit(n,i)
print(f'after:  {n} = {bin(n)}')

before: 23 = 0b10111
after:  31 = 0b11111


##  2.3. Clear $i^{th}$ bit


In [37]:
def clear_ith_bit(n:int, i:int)->int:
    "return n by clearing the i-th bit"
    mask = 1<<i
    return n & ~(mask)

In [44]:
# test 
n = 23
i = 2
print(f'before: {n} = {bin(n)}')

n = clear_ith_bit(n,i)
print(f'after:  {n} = {bin(n)}')

before: 23 = 0b10111
after:  19 = 0b10011


# 3. Additional manipulations
## 3.1. Update $i^{th}$ bit 

Update the i-th bit as per given value

In [45]:
def update_ith_bit(n:int, i:int, val:bool) -> int:
    """return n by setting the its ith bit with value"""
    mask = 1<<i # 1 followed by i 0s
    return n | mask if val == True else n &  ~mask 

In [48]:
n = 42
i = 0   # pos to set
j = 3   # pos to clear   

print(f'{n} = {bin(n)}')

n=update_ith_bit(n,i,True)
print(f'after setting bit {i}: {n} ({bin(n)})')

n=update_ith_bit(n,j,False)
print(f'after clearing bit {j}: {n} ({bin(n)})')

42 = 0b101010
after setting bit 0: 43 (0b101011)
after clearing bit 3: 35 (0b100011)


## 3.2. Clear the last $i$ bits 

In [57]:
def clear_last_i_bits(n:int,i:int)->int:
    """returns n by clearing its last i bits"""
    return n & (~0<<i)   # ~0 = all 1, lshifted by i 

In [58]:
n = 93
i = 4

result = clear_last_i_bits(n,i)
print(f'n={bin(n)}, i={4}, result={bin(result)}')

n=0b1011101, i=4, result=0b1010000


## 3.3. Clear a range of bits 
Clear the bits between indices $i$ and $j$. 

Soln:
1. `-1`: all `1`
2. `-1 << i+1` : clear all bits of all 1 bitstring from index `i` (`1...10...0`)
3. `~(-1 << j)`: set the last `j` bits with leading `0` (`0...01...1`)
4. `(-1 << i+1) | ~(-1 << j)` : only `[i:j]` window is cleared `1...10...01...1`

In [72]:
def clear_range(n:int, i:int, j:int)-> int:
    """ returns n by clearing bits between index i and j"""
    if i > j:
        mask = (-1 << i+1)|~(-1 << j)
        return n & mask
    raise Exception('IndexError') # call exeption if indices overlap

In [74]:
n = 255
i = 5
j = 2

result = clear_range(n,i,j)

print(f'n={bin(n)}, i={i}, j={j}, result={bin(result)}')

n=0b11111111, i=5, j=2, result=0b11000011


## 3.4. Inset bit string between indices 
Insert bitstring $m$ within the indices $i$ and $j$ of $n$

1. Clear the window $[i:j]$ of $n$ with _Clear Mask_ $CM$=`(-1 << (i+1)) | ~(-1 << j)`
2. LShift $m$ by $j$ to allign, lets call it _Insert Mask_ $IM$ = `m<<j`
3. Execute `(n & CM) | IM`


In [76]:
def insert_bitstring(n:int, m:int, i:int, j:int)->int:
    """inserts m witin n at window [i,j]
       assumes |m|=(i-j+1)
    """
    clear_mask = (-1 << (i+1)) | ~(-1 << j)
    insert_mask = m << j

    return (n & clear_mask) | insert_mask
    

In [80]:
n = 511  
m = 45
i = 7
j=2

result = insert_bitstring(n,m,i,j)

print(f'n={bin(n)}, m={bin(m)}, i={i}, j={j}, result={bin(result)}')

n=0b111111111, m=0b101101, i=7, j=2, result=0b110110111
